In [1]:
!cp -r "/content/drive/MyDrive/Ecole/Internship/data/FB15-K-237" "/content/"
!cp -r "/content/drive/MyDrive/Ecole/Internship/data/WN18RR" "/content/"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

import gensim
import gensim.downloader as api
from gensim.models import word2vec, KeyedVectors

In [ ]:
def clean_word(word):
    word = word.split('.')[0]
    word = word.replace('_', ' ')
    return word

In [8]:
df_wn_train = pd.read_csv('/content/WN18RR/text/train.txt', sep='\t', header=None)
df_wn_train.columns = ['word1', 'relation', 'word2']

df_wn_valid = pd.read_csv('/content/WN18RR/text/valid.txt', sep='\t', header=None)
df_wn_valid.columns = ['word1', 'relation', 'word2']

df_wn_test = pd.read_csv('/content/WN18RR/text/test.txt', sep='\t', header=None)
df_wn_test.columns = ['word1', 'relation', 'word2']

df_wn_train.head()

,word1,relation,word2
0,land_reform.n.01,_hypernym,reform.n.01
1,cover.v.01,_derivationally_related_form,covering.n.02
2,botany.n.02,_derivationally_related_form,botanize.v.01
3,kamet.n.01,_instance_hypernym,mountain_peak.n.01
4,question.n.01,_derivationally_related_form,ask.v.01


In [10]:
df_wn_train.shape, df_wn_valid.shape, df_wn_test.shape

((86835, 3), (3034, 3), (3134, 3))

In [11]:
df_wn_train['relation'].unique()

array(['_hypernym', '_derivationally_related_form', '_instance_hypernym',
       '_also_see', '_member_meronym', '_synset_domain_topic_of',
       '_has_part', '_member_of_domain_usage', '_member_of_domain_region',
       '_verb_group', '_similar_to'], dtype=object)

In [12]:
bad_relations = ['_derivationally_related_form', ] # '_verb_group'?
df_wn_train = df_wn_train[~df_wn_train['relation'].isin(bad_relations)]
df_wn_valid = df_wn_valid[~df_wn_valid['relation'].isin(bad_relations)]

In [14]:
df_wn_train['clean_word1'] = df_wn_train['word1'].apply(lambda x: clean_word(x))
df_wn_train['clean_word2'] = df_wn_train['word2'].apply(lambda x: clean_word(x))

df_wn_valid['clean_word1'] = df_wn_valid['word1'].apply(lambda x: clean_word(x))
df_wn_valid['clean_word2'] = df_wn_valid['word2'].apply(lambda x: clean_word(x))
df_wn_train.head()

,word1,relation,word2,clean_word1,clean_word2
0,land_reform.n.01,_hypernym,reform.n.01,land reform,reform
3,kamet.n.01,_instance_hypernym,mountain_peak.n.01,kamet,mountain peak
6,hoist.v.03,_hypernym,raise.v.02,hoist,raise
7,empathy.n.01,_hypernym,sympathy.n.02,empathy,sympathy
8,disengagement.n.02,_hypernym,retreat.n.01,disengagement,retreat


In [15]:
le = LabelEncoder()
df_wn_train['relation_num'] = le.fit_transform(df_wn_train['relation'].values)
df_wn_valid['relation_num'] = le.transform(df_wn_valid['relation'].values)

ConceptNet Numberbatch

In [3]:
!wget https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-en-19.08.txt.gz

--2022-05-10 17:20:47--  https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-en-19.08.txt.gz
Resolving conceptnet.s3.amazonaws.com (conceptnet.s3.amazonaws.com)... 52.217.164.57
Connecting to conceptnet.s3.amazonaws.com (conceptnet.s3.amazonaws.com)|52.217.164.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 325403502 (310M) [application/x-gzip]
Saving to: ‘numberbatch-en-19.08.txt.gz’

numberbatch-en-19.0 100%[===================>] 310.33M  42.9MB/s    in 8.2s    

2022-05-10 17:20:56 (37.6 MB/s) - ‘numberbatch-en-19.08.txt.gz’ saved [325403502/325403502]



In [4]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)

In [5]:
!gunzip numberbatch-en-19.08.txt.gz
conceptnet_model = load_embeddings("numberbatch-en-19.08.txt")